In [8]:
import pandas as pd
import zipfile
import os
import time
from tqdm import tqdm
import shutil
import pandas as pd

import warnings
warnings.filterwarnings("ignore")
!apt-get install tshark
!pip install scapy



Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libbcg729-0 libc-ares2 liblua5.2-0 libmaxminddb0 libnl-genl-3-200 libpcap0.8 libsbc1 libsmi2ldbl
  libspandsp2 libspeexdsp1 libwireshark-data libwireshark15 libwiretap12 libwsutil13
  wireshark-common
Suggested packages:
  mmdb-bin snmp-mibs-downloader geoipupdate geoip-database geoip-database-extra libjs-leaflet
  libjs-leaflet.markercluster wireshark-doc
The following NEW packages will be installed:
  libbcg729-0 libc-ares2 liblua5.2-0 libmaxminddb0 libnl-genl-3-200 libpcap0.8 libsbc1 libsmi2ldbl
  libspandsp2 libspeexdsp1 libwireshark-data libwireshark15 libwiretap12 libwsutil13 tshark
  wireshark-common
0 upgraded, 16 newly installed, 0 to remove and 49 not upgraded.
Need to get 23.0 MB of archives.
After this operation, 120 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 libmaxmindd

In [9]:
def folder(f_name): #this function creates a folder named "attacks" in the program directory.
    try:
        if not os.path.exists(f_name):
            os.makedirs(f_name)
    except OSError:
        print ("The folder could not be created!")

In [27]:
def find_the_way(path,file_format,con=""):
    files_add = []
    # r=root, d=directories, f = files
    for r, d, f in os.walk(path):
        for file in f:
            if file_format in file:
                if con in file:
                    files_add.append(os.path.join(r, file))

    return files_add
path="pcaps"
files_add=find_the_way(path,'.pcap')
files_add

['pcaps/scan-hostport-4-dec.pcap',
 'pcaps/scan-portos-1-dec.pcap',
 'pcaps/dos-synflooding-2-dec.pcap',
 'pcaps/scan-hostport-3-dec.pcap',
 'pcaps/mitm-arpspoofing-4-dec.pcap',
 'pcaps/scan-hostport-1-dec.pcap',
 'pcaps/mirai-ackflooding-4-dec.pcap',
 'pcaps/scan-portos-4-dec.pcap',
 'pcaps/mitm-arpspoofing-5-dec.pcap',
 'pcaps/scan-hostport-2-dec.pcap',
 'pcaps/scan-portos-6-dec.pcap',
 'pcaps/dos-synflooding-4-dec.pcap',
 'pcaps/mirai-udpflooding-3-dec.pcap',
 'pcaps/dos-synflooding-5-dec.pcap',
 'pcaps/mirai-hostbruteforce-2-dec.pcap',
 'pcaps/mitm-arpspoofing-3-dec.pcap',
 'pcaps/scan-hostport-6-dec.pcap',
 'pcaps/mirai-ackflooding-2-dec.pcap',
 'pcaps/mitm-arpspoofing-1-dec.pcap',
 'pcaps/mirai-httpflooding-3-dec.pcap',
 'pcaps/scan-portos-2-dec.pcap',
 'pcaps/mirai-hostbruteforce-4-dec.pcap',
 'pcaps/mirai-udpflooding-2-dec.pcap',
 'pcaps/scan-hostport-5-dec.pcap',
 'pcaps/mirai-hostbruteforce-5-dec.pcap',
 'pcaps/scan-portos-5-dec.pcap',
 'pcaps/mirai-hostbruteforce-3-dec.pcap'

In [11]:
def flag_fixer(file):

    IP_flags = {'0': 1, '<Flag 0 ()>': 2, '<Flag 2 (DF)>': 3, '<Flag 1 (MF)>': 4,'<Flag 3 (MF+DF)>':5,
     '<Flag 4 (evil)>':6,
     '<Flag 5 (MF+evil)>':7,
     '<Flag 6 (DF+evil)>':8,
     '<Flag 7 (MF+DF+evil)>':9,
     '<Flag 3 (MF+DF)>':10,
     '<Flag 4 (evil)>':11,
     '<Flag 5 (MF+evil)>':12,
     '<Flag 6 (DF+evil)>':13,
     '<Flag 7 (MF+DF+evil)>':14,
     '<Flag 3 (MF+DF)>':15,
     '<Flag 4 (evil)>':16,
     '<Flag 5 (MF+evil)>':17,
     '<Flag 6 (DF+evil)>':18,
     '<Flag 7 (MF+DF+evil)>':19,
     '<Flag 3 (MF+DF)>':20,
     '<Flag 4 (evil)>':21,
     '<Flag 5 (MF+evil)>':22,
     '<Flag 6 (DF+evil)>':23,
     '<Flag 7 (MF+DF+evil)>':24,
     '<Flag 3 (MF+DF)>':25,
     '<Flag 4 (evil)>':26,
     '<Flag 5 (MF+evil)>':27,
     '<Flag 6 (DF+evil)>':28,
     '<Flag 7 (MF+DF+evil)>':29,
     '<Flag 3 (MF+DF)>':30,
     '<Flag 4 (evil)>':31,
     '<Flag 5 (MF+evil)>':32,
     '<Flag 6 (DF+evil)>':33,
     '<Flag 7 (MF+DF+evil)>':34,
     '<Flag 3 (MF+DF)>':35,
     '<Flag 4 (evil)>':36,
     '<Flag 5 (MF+evil)>':37,
     '<Flag 6 (DF+evil)>':38,
     '<Flag 7 (MF+DF+evil)>':39,
     '<Flag 3 (MF+DF)>':40,
     '<Flag 4 (evil)>':41,
     '<Flag 5 (MF+evil)>':42,
     '<Flag 6 (DF+evil)>':43,
     '<Flag 7 (MF+DF+evil)>':44}
    TCP_flags = {'0': 1, '<Flag 2 (S)>': 2, '<Flag 18 (SA)>': 3, '<Flag 16 (A)>': 4, '<Flag 24 (PA)>': 5, '<Flag 25 (FPA)>': 6, '<Flag 17 (FA)>': 7, '<Flag 4 (R)>': 8, '<Flag 20 (RA)>': 9, '<Flag 194 (SEC)>': 10, '<Flag 1 (F)>': 11, '<Flag 152 (PAC)>': 12, '<Flag 144 (AC)>': 13,'<Flag 82 (SAE)>':14,'<Flag 49 (FAU)>':15, '<Flag 0 ()>':16,
     '<Flag 10 (SP)>':17,
     '<Flag 100 (RUE)>':18,
     '<Flag 102 (SRUE)>':19,
     '<Flag 103 (FSRUE)>':20,
     '<Flag 104 (PUE)>':21,
     '<Flag 106 (SPUE)>':22,
     '<Flag 108 (RPUE)>':23,
     '<Flag 11 (FSP)>':24,
     '<Flag 110 (SRPUE)>':25,
     '<Flag 111 (FSRPUE)>':26,
     '<Flag 112 (AUE)>':27,
     '<Flag 113 (FAUE)>':28,
     '<Flag 114 (SAUE)>':29,
     '<Flag 116 (RAUE)>':30,
     '<Flag 117 (FRAUE)>':31,
     '<Flag 119 (FSRAUE)>':32,
     '<Flag 121 (FPAUE)>':33,
     '<Flag 123 (FSPAUE)>':34,
     '<Flag 124 (RPAUE)>':35,
     '<Flag 125 (FRPAUE)>':36,
     '<Flag 126 (SRPAUE)>':37,
     '<Flag 127 (FSRPAUE)>':38,
     '<Flag 128 (C)>':39,
     '<Flag 129 (FC)>':40,
     '<Flag 13 (FRP)>':41,
     '<Flag 130 (SC)>':42,
     '<Flag 131 (FSC)>':43,
     '<Flag 133 (FRC)>':44,
     '<Flag 134 (SRC)>':45,
     '<Flag 135 (FSRC)>':46,
     '<Flag 136 (PC)>':47,
     '<Flag 137 (FPC)>':48,
     '<Flag 138 (SPC)>':49,
     '<Flag 139 (FSPC)>':50,
     '<Flag 14 (SRP)>':51,
     '<Flag 140 (RPC)>':52,
     '<Flag 145 (FAC)>':53,
     '<Flag 146 (SAC)>':54,
     '<Flag 147 (FSAC)>':55,
     '<Flag 148 (RAC)>':56,
     '<Flag 150 (SRAC)>':57,
     '<Flag 151 (FSRAC)>':58,
     '<Flag 153 (FPAC)>':59,
     '<Flag 154 (SPAC)>':60,
     '<Flag 155 (FSPAC)>':61,
     '<Flag 156 (RPAC)>':62,
     '<Flag 157 (FRPAC)>':63,
     '<Flag 158 (SRPAC)>':64,
     '<Flag 159 (FSRPAC)>':65,
     '<Flag 160 (UC)>':66,
     '<Flag 162 (SUC)>':67,
     '<Flag 163 (FSUC)>':68,
     '<Flag 164 (RUC)>':69,
     '<Flag 166 (SRUC)>':70,
     '<Flag 169 (FPUC)>':71,
     '<Flag 170 (SPUC)>':72,
     '<Flag 171 (FSPUC)>':73,
     '<Flag 172 (RPUC)>':74,
     '<Flag 174 (SRPUC)>':75,
     '<Flag 176 (AUC)>':76,
     '<Flag 177 (FAUC)>':77,
     '<Flag 179 (FSAUC)>':78,
     '<Flag 180 (RAUC)>':79,
     '<Flag 181 (FRAUC)>':80,
     '<Flag 182 (SRAUC)>':81,
     '<Flag 185 (FPAUC)>':82,
     '<Flag 186 (SPAUC)>':83,
     '<Flag 187 (FSPAUC)>':84,
     '<Flag 188 (RPAUC)>':85,
     '<Flag 190 (SRPAUC)>':86,
     '<Flag 191 (FSRPAUC)>':87,
     '<Flag 195 (FSEC)>':88,
     '<Flag 197 (FREC)>':89,
     '<Flag 199 (FSREC)>':90,
     '<Flag 200 (PEC)>':91,
     '<Flag 201 (FPEC)>':92,
     '<Flag 202 (SPEC)>':93,
     '<Flag 203 (FSPEC)>':94,
     '<Flag 204 (RPEC)>':95,
     '<Flag 205 (FRPEC)>':96,
     '<Flag 206 (SRPEC)>':97,
     '<Flag 207 (FSRPEC)>':98,
     '<Flag 208 (AEC)>':99,
     '<Flag 209 (FAEC)>':100,
     '<Flag 210 (SAEC)>':101,
     '<Flag 212 (RAEC)>':102,
     '<Flag 213 (FRAEC)>':103,
     '<Flag 214 (SRAEC)>':104,
     '<Flag 216 (PAEC)>':105,
     '<Flag 218 (SPAEC)>':106,
     '<Flag 219 (FSPAEC)>':107,
     '<Flag 22 (SRA)>':108,
     '<Flag 220 (RPAEC)>':109,
     '<Flag 222 (SRPAEC)>':110,
     '<Flag 224 (UEC)>':111,
     '<Flag 228 (RUEC)>':112,
     '<Flag 229 (FRUEC)>':113,
     '<Flag 23 (FSRA)>':114,
     '<Flag 232 (PUEC)>':115,
     '<Flag 233 (FPUEC)>':116,
     '<Flag 234 (SPUEC)>':117,
     '<Flag 236 (RPUEC)>':118,
     '<Flag 237 (FRPUEC)>':119,
     '<Flag 238 (SRPUEC)>':120,
     '<Flag 240 (AUEC)>':121,
     '<Flag 241 (FAUEC)>':122,
     '<Flag 242 (SAUEC)>':123,
     '<Flag 244 (RAUEC)>':124,
     '<Flag 246 (SRAUEC)>':125,
     '<Flag 248 (PAUEC)>':126,
     '<Flag 249 (FPAUEC)>':127,
     '<Flag 251 (FSPAUEC)>':128,
     '<Flag 252 (RPAUEC)>':129,
     '<Flag 253 (FRPAUEC)>':130,
     '<Flag 254 (SRPAUEC)>':131,
     '<Flag 255 (FSRPAUEC)>':132,
     '<Flag 256 (N)>':133,
     '<Flag 257 (FN)>':134,
     '<Flag 258 (SN)>':135,
     '<Flag 259 (FSN)>':136,
     '<Flag 26 (SPA)>':137,
     '<Flag 260 (RN)>':138,
     '<Flag 261 (FRN)>':139,
     '<Flag 262 (SRN)>':140,
     '<Flag 263 (FSRN)>':141,
     '<Flag 264 (PN)>':142,
     '<Flag 267 (FSPN)>':143,
     '<Flag 268 (RPN)>':144,
     '<Flag 270 (SRPN)>':145,
     '<Flag 272 (AN)>':146,
     '<Flag 273 (FAN)>':147,
     '<Flag 274 (SAN)>':148,
     '<Flag 275 (FSAN)>':149,
     '<Flag 276 (RAN)>':150,
     '<Flag 278 (SRAN)>':151,
     '<Flag 28 (RPA)>':152,
     '<Flag 280 (PAN)>':153,
     '<Flag 281 (FPAN)>':154,
     '<Flag 282 (SPAN)>':155,
     '<Flag 283 (FSPAN)>':156,
     '<Flag 285 (FRPAN)>':157,
     '<Flag 286 (SRPAN)>':158,
     '<Flag 287 (FSRPAN)>':159,
     '<Flag 288 (UN)>':160,
     '<Flag 29 (FRPA)>':161,
     '<Flag 291 (FSUN)>':162,
     '<Flag 292 (RUN)>':163,
     '<Flag 293 (FRUN)>':164,
     '<Flag 296 (PUN)>':165,
     '<Flag 297 (FPUN)>':166,
     '<Flag 298 (SPUN)>':167,
     '<Flag 3 (FS)>':168,
     '<Flag 30 (SRPA)>':169,
     '<Flag 301 (FRPUN)>':170,
     '<Flag 302 (SRPUN)>':171,
     '<Flag 306 (SAUN)>':172,
     '<Flag 307 (FSAUN)>':173,
     '<Flag 308 (RAUN)>':174,
     '<Flag 309 (FRAUN)>':175,
     '<Flag 31 (FSRPA)>':176,
     '<Flag 311 (FSRAUN)>':177,
     '<Flag 312 (PAUN)>':178,
     '<Flag 313 (FPAUN)>':179,
     '<Flag 314 (SPAUN)>':180,
     '<Flag 315 (FSPAUN)>':181,
     '<Flag 316 (RPAUN)>':182,
     '<Flag 318 (SRPAUN)>':183,
     '<Flag 32 (U)>':184,
     '<Flag 323 (FSEN)>':185,
     '<Flag 324 (REN)>':186,
     '<Flag 325 (FREN)>':187,
     '<Flag 326 (SREN)>':188,
     '<Flag 329 (FPEN)>':189,
     '<Flag 33 (FU)>':190,
     '<Flag 330 (SPEN)>':191,
     '<Flag 332 (RPEN)>':192,
     '<Flag 333 (FRPEN)>':193,
     '<Flag 335 (FSRPEN)>':194,
     '<Flag 336 (AEN)>':195,
     '<Flag 338 (SAEN)>':196,
     '<Flag 339 (FSAEN)>':197,
     '<Flag 34 (SU)>':198,
     '<Flag 340 (RAEN)>':199,
     '<Flag 341 (FRAEN)>':200,
     '<Flag 342 (SRAEN)>':201,
     '<Flag 343 (FSRAEN)>':202,
     '<Flag 344 (PAEN)>':203,
     '<Flag 347 (FSPAEN)>':204,
     '<Flag 348 (RPAEN)>':205,
     '<Flag 349 (FRPAEN)>':206,
     '<Flag 350 (SRPAEN)>':207,
     '<Flag 351 (FSRPAEN)>':208,
     '<Flag 352 (UEN)>':209,
     '<Flag 353 (FUEN)>':210,
     '<Flag 354 (SUEN)>':211,
     '<Flag 355 (FSUEN)>':212,
     '<Flag 356 (RUEN)>':213,
     '<Flag 357 (FRUEN)>':214,
     '<Flag 358 (SRUEN)>':215,
     '<Flag 359 (FSRUEN)>':216,
     '<Flag 36 (RU)>':217,
     '<Flag 361 (FPUEN)>':218,
     '<Flag 365 (FRPUEN)>':219,
     '<Flag 366 (SRPUEN)>':220,
     '<Flag 368 (AUEN)>':221,
     '<Flag 369 (FAUEN)>':222,
     '<Flag 37 (FRU)>':223,
     '<Flag 370 (SAUEN)>':224,
     '<Flag 372 (RAUEN)>':225,
     '<Flag 374 (SRAUEN)>':226,
     '<Flag 376 (PAUEN)>':227,
     '<Flag 377 (FPAUEN)>':228,
     '<Flag 378 (SPAUEN)>':229,
     '<Flag 379 (FSPAUEN)>':230,
     '<Flag 38 (SRU)>':231,
     '<Flag 380 (RPAUEN)>':232,
     '<Flag 383 (FSRPAUEN)>':233,
     '<Flag 384 (CN)>':234,
     '<Flag 385 (FCN)>':235,
     '<Flag 389 (FRCN)>':236,
     '<Flag 390 (SRCN)>':237,
     '<Flag 391 (FSRCN)>':238,
     '<Flag 392 (PCN)>':239,
     '<Flag 393 (FPCN)>':240,
     '<Flag 394 (SPCN)>':241,
     '<Flag 396 (RPCN)>':242,
     '<Flag 398 (SRPCN)>':243,
     '<Flag 40 (PU)>':244,
     '<Flag 401 (FACN)>':245,
     '<Flag 402 (SACN)>':246,
     '<Flag 403 (FSACN)>':247,
     '<Flag 404 (RACN)>':248,
     '<Flag 405 (FRACN)>':249,
     '<Flag 406 (SRACN)>':250,
     '<Flag 408 (PACN)>':251,
     '<Flag 409 (FPACN)>':252,
     '<Flag 41 (FPU)>':253,
     '<Flag 410 (SPACN)>':254,
     '<Flag 411 (FSPACN)>':255,
     '<Flag 412 (RPACN)>':256,
     '<Flag 413 (FRPACN)>':257,
     '<Flag 415 (FSRPACN)>':258,
     '<Flag 418 (SUCN)>':259,
     '<Flag 419 (FSUCN)>':260,
     '<Flag 42 (SPU)>':261,
     '<Flag 420 (RUCN)>':262,
     '<Flag 421 (FRUCN)>':263,
     '<Flag 422 (SRUCN)>':264,
     '<Flag 424 (PUCN)>':265,
     '<Flag 426 (SPUCN)>':266,
     '<Flag 427 (FSPUCN)>':267,
     '<Flag 43 (FSPU)>':268,
     '<Flag 433 (FAUCN)>':269,
     '<Flag 436 (RAUCN)>':270,
     '<Flag 437 (FRAUCN)>':271,
     '<Flag 438 (SRAUCN)>':272,
     '<Flag 44 (RPU)>':273,
     '<Flag 440 (PAUCN)>':274,
     '<Flag 442 (SPAUCN)>':275,
     '<Flag 444 (RPAUCN)>':276,
     '<Flag 445 (FRPAUCN)>':277,
     '<Flag 446 (SRPAUCN)>':278,
     '<Flag 447 (FSRPAUCN)>':279,
     '<Flag 448 (ECN)>':280,
     '<Flag 449 (FECN)>':281,
     '<Flag 450 (SECN)>':282,
     '<Flag 451 (FSECN)>':283,
     '<Flag 455 (FSRECN)>':284,
     '<Flag 456 (PECN)>':285,
     '<Flag 458 (SPECN)>':286,
     '<Flag 46 (SRPU)>':287,
     '<Flag 463 (FSRPECN)>':288,
     '<Flag 464 (AECN)>':289,
     '<Flag 465 (FAECN)>':290,
     '<Flag 466 (SAECN)>':291,
     '<Flag 467 (FSAECN)>':292,
     '<Flag 468 (RAECN)>':293,
     '<Flag 469 (FRAECN)>':294,
     '<Flag 47 (FSRPU)>':295,
     '<Flag 472 (PAECN)>':296,
     '<Flag 474 (SPAECN)>':297,
     '<Flag 475 (FSPAECN)>':298,
     '<Flag 476 (RPAECN)>':299,
     '<Flag 478 (SRPAECN)>':300,
     '<Flag 48 (AU)>':301,
     '<Flag 482 (SUECN)>':302,
     '<Flag 483 (FSUECN)>':303,
     '<Flag 484 (RUECN)>':304,
     '<Flag 485 (FRUECN)>':305,
     '<Flag 489 (FPUECN)>':306,
     '<Flag 491 (FSPUECN)>':307,
     '<Flag 492 (RPUECN)>':308,
     '<Flag 493 (FRPUECN)>':309,
     '<Flag 495 (FSRPUECN)>':310,
     '<Flag 497 (FAUECN)>':311,
     '<Flag 498 (SAUECN)>':312,
     '<Flag 5 (FR)>':313,
     '<Flag 50 (SAU)>':314,
     '<Flag 502 (SRAUECN)>':315,
     '<Flag 503 (FSRAUECN)>':316,
     '<Flag 505 (FPAUECN)>':317,
     '<Flag 507 (FSPAUECN)>':318,
     '<Flag 508 (RPAUECN)>':319,
     '<Flag 51 (FSAU)>':320,
     '<Flag 510 (SRPAUECN)>':321,
     '<Flag 511 (FSRPAUECN)>':322,
     '<Flag 52 (RAU)>':323,
     '<Flag 55 (FSRAU)>':324,
     '<Flag 56 (PAU)>':325,
     '<Flag 57 (FPAU)>':326,
     '<Flag 58 (SPAU)>':327,
     '<Flag 59 (FSPAU)>':328,
     '<Flag 6 (SR)>':329,
     '<Flag 61 (FRPAU)>':330,
     '<Flag 62 (SRPAU)>':331,
     '<Flag 66 (SE)>':332,
     '<Flag 67 (FSE)>':333,
     '<Flag 68 (RE)>':334,
     '<Flag 7 (FSR)>':335,
     '<Flag 72 (PE)>':336,
     '<Flag 73 (FPE)>':337,
     '<Flag 74 (SPE)>':338,
     '<Flag 75 (FSPE)>':339,
     '<Flag 76 (RPE)>':340,
     '<Flag 77 (FRPE)>':341,
     '<Flag 79 (FSRPE)>':342,
     '<Flag 8 (P)>':343,
     '<Flag 80 (AE)>':344,
     '<Flag 81 (FAE)>':345,
     '<Flag 83 (FSAE)>':346,
     '<Flag 84 (RAE)>':347,
     '<Flag 85 (FRAE)>':348,
     '<Flag 86 (SRAE)>':349,
     '<Flag 87 (FSRAE)>':350,
     '<Flag 88 (PAE)>':351,
     '<Flag 89 (FPAE)>':352,
     '<Flag 90 (SPAE)>':353,
     '<Flag 92 (RPAE)>':354,
     '<Flag 93 (FRPAE)>':355,
     '<Flag 94 (SRPAE)>':356,
     '<Flag 95 (FSRPAE)>':357,
     '<Flag 96 (UE)>':358,
     '<Flag 98 (SUE)>':359,
     '<Flag 99 (FSUE)>':360}
    BOOTP_flags = {'0': 1, '<Flag 0 ()>': 2, '<Flag 32768 (B)>': 3, 0: 1}
    Protocol = {'EAPOL': 1, 'DHCP': 2, 'DNS': 3, 'TCP': 4, 'HTTP': 5, 'ICMP': 6, 'MDNS': 7, 'IGMPv3': 8, 'SSDP': 9, 'NTP': 10, 'HTTP/XML': 11, 'UDP': 12, 'SSLv2': 13, 'TLSv1': 14, 'ADwin Config': 15, 'TLSv1.2': 16, 'ICMPv6': 17, 'HTTP/JSON': 18, 'XID': 19, 'TFTP': 20, 'NXP 802.15.4 SNIFFER': 21, 'IGMPv2': 22, 'A21': 23, 'STUN': 24, 'Gearman': 25, '? KNXnet/IP': 26, 'UDPENCAP': 27, 'ESP': 28, 'SSL': 29, 'NBNS': 30, 'SIP': 31, 'BROWSER': 32, 'SABP': 33, 'ISAKMP': 34, 'CLASSIC-STUN': 35, 'Omni-Path': 36, 'XMPP/XML': 37, 'ULP': 38, 'TFP over TCP': 39, 'AX4000': 40, 'MIH': 41, 'DHCPv6': 42, 'TDLS': 43, 'RTMP': 44, 'TCPCL': 45, 'IPA': 46, 'GQUIC': 47, '0x86dd': 48, 'DB-LSP-DISC': 49, 'SSLv3': 50, 'LLMNR': 51, 'FB_ZERO': 52, 'OCSP': 53, 'IPv4': 54, 'STP': 55, 'SSH': 56, 'TLSv1.1': 57, 'KINK': 58, 'MANOLITO': 59, 'PKTC': 60, 'TELNET': 61, 'RTSP': 62, 'HCrt': 63, 'MPTCP': 64, 'S101': 65, 'IRC': 66, 'AJP13': 67, 'PMPROXY': 68, 'PNIO': 69, 'AMS': 70, 'ECATF': 71, 'LLC': 72, 'TZSP': 73,'RSIP':74,'SSHv2':75
    ,'DIAMETER':76
    ,'BFD Control':77
    ,'ASAP':78
    ,'DISTCC':79
    ,'DISTCC ':79
    ,'LISP':80
    ,'WOW':81
    ,'DTLSv1.0':82
    ,'SNMP':83
    ,'SMB2':84
    ,'SMB':85
    ,'NBSS':86
    ,'UDT':87,'HiQnet':88
    ,'POWERLINK/UDP':89
    ,'RTP':90
    ,'WebSocket':91
    ,'NAT-PMP':92
    ,'RTCP':93,'Syslog':94
    ,'Portmap':95
    ,'OpenVPN':96
    ,'BJNP':97
    ,'RIPv1':98
    ,'MAC-Telnet':99
    ,'ECHO':100
    ,'ASF':101
    ,'DAYTIME':102
    ,'SRVLOC':103
    ,'KRB4':104
    ,'CAPWAP-Control':105
    ,'XDMCP':106
    ,'Chargen':107
    ,'RADIUS':108
    ,'L2TP':109
    ,'DCERPC':110
    ,'KPASSWD':111
    ,'H264':112
    ,'FTP':113
    ,'FTP-DATA':114
    ,'ENIP':115
    ,'RIPv2':116
    ,'ICP':117,
    "BACnet-APDU":118,
    "IAX2":119,
    "RX":120,
    "HTTP2":121,
    "SIP/SDP":122,
    "TIME":123,
    "Elasticsearch":124,
    "RSL":125,
    "TPCP":126,
     "IPv6":  127,"ARP":  128 }

    df=pd.read_csv(file)
    IP_flags

    df["IP_flags"]=df["IP_flags"].map(IP_flags.get)
    df["TCP_flags"]=df["TCP_flags"].map(TCP_flags.get)
    df["BOOTP_flags"]=df["BOOTP_flags"].map(BOOTP_flags.get)
    df["Protocol"]=df["Protocol"].map(Protocol.get)

    #df=df.replace({"IP_flags": IP_flags})
    #df=df.replace({"TCP_flags": TCP_flags})
    #df=df.replace({"BOOTP_flags": BOOTP_flags})
    #df=df.replace({"Protocol": Protocol})
    #df=df.replace({"Label": label})
    df.to_csv(file,index=None)





# PCAP2CSV

In [29]:
from scapy.all import*

In [30]:
import math
import pandas as pd
import os
import numpy as np

In [31]:
def folder(f_name): #this function creates a folder.
    try:
        if not os.path.exists(f_name):
            os.makedirs(f_name)
    except OSError:
        print ("The folder could not be created!")

In [32]:
def shannon(data):
    LOG_BASE = 2
   # We determine the frequency of each byte
   # in the dataset and if this frequency is not null we use it for the
   # entropy calculation
    dataSize = len(data)
    ent = 0.0
    freq={}
    for c in data:
        if c in freq:
            freq[c] += 1
        else:
            freq[c] = 1
   # to determine if each possible value of a byte is in the list
    for key in freq.keys():
        f = float(freq[key])/dataSize
        if f > 0: # to avoid an error for log(0)
            ent = ent + f * math.log(f, LOG_BASE)
    return -ent

def pre_entropy(payload):

    characters=[]
    for i in payload:
            characters.append(i)
    return shannon(characters)


In [33]:
def port_class(port):
    port_list=[0,53,67,68,80,123,443,1900,5353,49153]# private port list (0-Reserved,53-DNS, 67-BOOTP server, 68-BOOTP client...)
    if port in port_list: #Is the port number in the list?
        return port_list.index(port)+1 # return the port's index number in the list (actually with index+1)
    elif 0 <= port <= 1023: # return 11 if the port number is in the range 0 :1023
        return 11
    elif  1024 <= port <= 49151 : # return 12 if the port number is in the range 1024:49151
        return 12
    elif 49152 <=port <= 65535 :# return 13 if the port number is in the range 49152:65535
        return 13
    else:# return 0 if no previous conditions are met
        return 0


def port_1023(port):
    if 0 <= port <= 1023:
        return port
    elif  1024 <= port <= 49151 :
        return 2
    elif 49152 <=port <= 65535 :
        return 3
    else:
        return 0

In [34]:
header="ts,Ether_dst,Ether_src,IP_src,IP_dst,pck_size,Ether_type,LLC_dsap,LLC_ssap,LLC_ctrl,EAPOL_version,EAPOL_type,EAPOL_len,IP_version,IP_ihl,IP_tos,IP_len,IP_flags,IP_Z,IP_MF,IP_id,IP_chksum,IP_DF,IP_frag,IP_ttl,IP_proto,IP_options,ICMP_type,ICMP_code,ICMP_chksum,ICMP_id,ICMP_seq,ICMP_ts_ori,ICMP_ts_rx,ICMP_ts_tx,ICMP_ptr,ICMP_reserved,ICMP_length,ICMP_nexthopmtu,ICMP_unused,TCP_seq,TCP_ack,TCP_dataofs,TCP_reserved,TCP_flags,TCP_FIN,TCP_SYN,TCP_RST,TCP_PSH,TCP_ACK,TCP_URG,TCP_ECE,TCP_CWR,TCP_window,TCP_chksum,TCP_urgptr,TCP_options,UDP_len,UDP_chksum,DHCP_options,BOOTP_op,BOOTP_htype,BOOTP_hlen,BOOTP_hops,BOOTP_xid,BOOTP_secs,BOOTP_flags,BOOTP_sname,BOOTP_file,BOOTP_options,DNS_length,DNS_id,DNS_qr,DNS_opcode,DNS_aa,DNS_tc,DNS_rd,DNS_ra,DNS_z,DNS_ad,DNS_cd,DNS_rcode,DNS_qdcount,DNS_ancount,DNS_nscount,DNS_arcount,sport_class,dport_class,sport23,dport23,sport_bare,dport_bare,TCP_sport,TCP_dport,UDP_sport,UDP_dport,payload_bytes,entropy,Label\n"

#header="pck_size,Ether_type,LLC_dsap,LLC_ssap,LLC_ctrl,EAPOL_version,EAPOL_type,EAPOL_len,IP_version,IP_ihl,IP_tos,IP_len,IP_flags,IP_frag,IP_ttl,IP_proto,IP_options,IP_add_count,ICMP_type,ICMP_code,ICMP_seq,ICMP_ts_ori,ICMP_ts_rx,ICMP_ts_tx,ICMP_gw,ICMP_ptr,ICMP_reserved,ICMP_length,ICMP_nexthopmtu,ICMP_unused,TCP_dataofs,TCP_reserved,TCP_flags,TCP_window,TCP_urgptr,UDP_len,BOOTP_op,BOOTP_htype,BOOTP_hlen,BOOTP_hops,BOOTP_secs,BOOTP_flags,DNS_length,DNS_qr,DNS_opcode,DNS_aa,DNS_tc,DNS_rd,DNS_ra,DNS_z,DNS_ad,DNS_cd,DNS_rcode,DNS_qdcount,DNS_ancount,DNS_nscount,DNS_arcount,sport,dport,entropy,Label,MAC\n"

In [35]:
#flags
#TCP
FIN = 0x01
SYN = 0x02
RST = 0x04
PSH = 0x08
ACK = 0x10
URG = 0x20
ECE = 0x40
CWR = 0x80
#IP
Z = 0x00
MF= 0x01
DF= 0x02


# Individual paket pased faetures

In [36]:
def pcap_parser(oldname):
    piece_size=50000
    new_name=oldname.replace(".pcap","_smaller.pcap")
    command='C:/\"Program Files\"/Wireshark/editcap.exe -c '+str(piece_size)+" \""+oldname+"\" "+new_name
    os.system(command)
    parsed=find_the_way("./",new_name[6:-5])
    return parsed,new_name[6:-5]


In [37]:
def merged_csv(name,keyword):
    for merger in ["_FE.csv","_WS.csv"]:
        merged_name=f"{name[:-4]}{merger}"
        csv_files=find_the_way("./",keyword,merger)
        df=pd.read_csv(csv_files[0])
        col_names=list(df.columns)

        empty = pd.DataFrame(columns=col_names)
        empty.to_csv(merged_name, mode="w", index=False)#,header=False)


        for j in csv_files:

            df=pd.read_csv(j)
            #print("name and shape of dataframe :",i,df.shape)
            df.to_csv(merged_name, mode="a", index=False,header=False)


            os.remove(j)
            try:
                os.remove(j.replace("_FE.csv",".pcap"))
            except:pass

            #print("\n\n\nname and shape of dataframe :",name,df.shape)

In [38]:
outputfolder="FE"
folder(outputfolder)
for sayac,i in enumerate (files_add):
    print(f"{sayac+1}/{len(files_add)}-{i}")
    try:
        csv=i.replace(".pcap",".csv")
        df=pd.read_csv(csv)
        df=df["Label"].values
        label=True
    except:
        print("Predetermined tag not found")
        label=False
    boyut=os.path.getsize(i)
    label_count=0
    if boyut >100000000:
        file_list,keyword=pcap_parser(i)

    else:
        file_list=[i]
    for isim in file_list:
        filename=isim.replace(".pcap","_FE.csv")
        ths = open(filename, "w")
        ths.write(header)


        pkt = rdpcap(isim)
        for j in tqdm(pkt):
            ts=float(j.time)
            try:pck_size=j.len
            except:pck_size=0
            if j.haslayer(Ether):


                Ether_dst=j[Ether].dst#Ether_adresses.index(j[Ether].dst)+1
                Ether_src=j[Ether].src#Ether_adj[Ether].dstresses.index(j[Ether].src)+1


                Ether_type=j[Ether].type
            else:
                Ether_dst=0
                Ether_src=0
                Ether_type=0


            if j.haslayer(ARP):
                ARP_hwtype=j[ARP].hwtype
                ARP_ptype=j[ARP].ptype
                ARP_hwlen=j[ARP].hwlen
                ARP_plen=j[ARP].plen
                ARP_op=j[ARP].op


                ARP_hwsrc=j[ARP].hwsrc
                ARP_psrc=j[ARP].psrc
                ARP_hwdst=j[ARP].hwdst
                ARP_pdst=j[ARP].pdst




                ARP_hwsrc=j[ARP].hwsrc#Ether_adresses.index(j[ARP].hwsrc)+1
                ARP_psrc=j[ARP].psrc#IP_adresses.index(j[ARP].psrc)+1
                ARP_hwdst=j[ARP].hwdst#Ether_adresses.index(j[ARP].hwdst)+1
                ARP_pdst=j[ARP].pdst#IP_adresses.index(j[ARP].pdst)+1


            else:
                ARP_hwtype=0
                ARP_ptype=0
                ARP_hwlen=0
                ARP_plen=0
                ARP_op=0
                ARP_hwsrc=0
                ARP_psrc=0
                ARP_hwdst=0
                ARP_pdst=0


            if j.haslayer(LLC):
                LLC_dsap=j[LLC].dsap
                LLC_ssap=j[LLC].ssap
                LLC_ctrl=j[LLC].ctrl
            else:
                LLC_dsap=0
                LLC_ssap=0
                LLC_ctrl=0



            if j.haslayer(EAPOL):
                EAPOL_version=j[EAPOL].version
                EAPOL_type=j[EAPOL].type
                EAPOL_len=j[EAPOL].len

            else:
                EAPOL_version=0
                EAPOL_type=0
                EAPOL_len=0


            if j.haslayer(IP):

                IP_Z = 0
                IP_MF= 0
                IP_DF= 0

                IP_version=j[IP].version
                IP_ihl=j[IP].ihl
                IP_tos=j[IP].tos
                IP_len=j[IP].len
                IP_id=j[IP].id
                IP_flags=j[IP].flags

                IP_frag=j[IP].frag
                IP_ttl=j[IP].ttl
                IP_proto=j[IP].proto
                IP_chksum=j[IP].chksum


                #if j[IP].options!=0:
                IP_options=j[IP].options
                if "IPOption_Router_Alert"   in str(IP_options):
                    IP_options=1
                else:IP_options=0



                #if IP_flags not in ipf: ipf.append(IP_flags)

                if IP_flags & Z:IP_Z = 1
                if IP_flags & MF:IP_MF = 1
                if IP_flags & DF:IP_DF = 1
                #if "Flag" in str(IP_flags):
                    #IP_flags=str(IP_flags)
                    #temp=IP_flags.find("(")
                    #IP_flags=int(IP_flags[6:temp-1])




                IP_src=j[IP].src#IP_adresses.index(j[IP].src)+1
                IP_dst=j[IP].dst#IP_adresses.index(j[IP].dst)+1



            else:
                IP_Z = 0
                IP_MF= 0
                IP_DF= 0

                IP_version=0
                IP_ihl=0
                IP_tos=0
                IP_len=0
                IP_id=0
                IP_flags=0
                IP_frag=0
                IP_ttl=0
                IP_proto=0
                IP_chksum=0
                IP_src=0
                IP_dst=0
                IP_options=0
                IP_add_count=0

            if j.haslayer(ICMP):
                ICMP_type=j[ICMP].type
                ICMP_code=j[ICMP].code
                ICMP_chksum=j[ICMP].chksum
                ICMP_id=j[ICMP].id
                ICMP_seq=j[ICMP].seq
                ICMP_ts_ori=j[ICMP].ts_ori
                ICMP_ts_rx=j[ICMP].ts_rx
                ICMP_ts_tx=j[ICMP].ts_tx
                ICMP_gw=j[ICMP].gw
                ICMP_ptr=j[ICMP].ptr
                ICMP_reserved=j[ICMP].reserved
                ICMP_length=j[ICMP].length
                ICMP_addr_mask=j[ICMP].addr_mask
                ICMP_nexthopmtu=j[ICMP].nexthopmtu
                ICMP_unused=j[ICMP].unused
            else:
                ICMP_type=0
                ICMP_code=0
                ICMP_chksum=0
                ICMP_id=0
                ICMP_seq=0
                ICMP_ts_ori=0
                ICMP_ts_rx=0
                ICMP_ts_tx=0
                ICMP_gw=0
                ICMP_ptr=0
                ICMP_reserved=0
                ICMP_length=0
                ICMP_addr_mask=0
                ICMP_nexthopmtu=0
                ICMP_unused=0




            if j.haslayer(TCP):
                TCP_FIN = 0
                TCP_SYN = 0
                TCP_RST = 0
                TCP_PSH = 0
                TCP_ACK = 0
                TCP_URG = 0
                TCP_ECE = 0
                TCP_CWR = 0
                TCP_sport=j[TCP].sport
                TCP_dport=j[TCP].dport
                TCP_seq=j[TCP].seq
                TCP_ack=j[TCP].ack
                TCP_dataofs=j[TCP].dataofs
                TCP_reserved=j[TCP].reserved
                TCP_flags=j[TCP].flags

                TCP_window=j[TCP].window
                TCP_chksum=j[TCP].chksum
                TCP_urgptr=j[TCP].urgptr
                TCP_options=j[TCP].options
                TCP_options= str(TCP_options).replace(",","-")
                if TCP_options!="0":
                    TCP_options=1
                else:
                    TCP_options=0




                #if TCP_flags not in tcpf:
                    #tcpf.append(TCP_flags)
                #print(TCP_options)
                if TCP_flags & FIN:TCP_FIN = 1
                if TCP_flags & SYN:TCP_SYN = 1
                if TCP_flags & RST:TCP_RST = 1
                if TCP_flags & PSH:TCP_PSH = 1
                if TCP_flags & ACK:TCP_ACK = 1
                if TCP_flags & URG:TCP_URG = 1
                if TCP_flags & ECE:TCP_ECE = 1
                if TCP_flags & CWR:TCP_CWR = 1
                #print(TCP_flags)
                #if "Flag" in str(TCP_flags):
                    #TCP_flags=str(TCP_flags)
                    #temp=TCP_flags.find("(")
                    #TCP_flags=int(TCP_flags[6:temp-1])




            else:
                TCP_sport=0
                TCP_dport=0
                TCP_seq=0
                TCP_ack=0
                TCP_dataofs=0
                TCP_reserved=0
                TCP_flags=0
                TCP_window=0
                TCP_chksum=0
                TCP_urgptr=0
                TCP_options=0
                TCP_options=0
                TCP_FIN = 0
                TCP_SYN = 0
                TCP_RST = 0
                TCP_PSH = 0
                TCP_ACK = 0
                TCP_URG = 0
                TCP_ECE = 0
                TCP_CWR = 0


            if j.haslayer(UDP):
                UDP_sport=j[UDP].sport
                UDP_dport=j[UDP].dport
                UDP_len=j[UDP].len
                UDP_chksum=j[UDP].chksum
            else:
                UDP_sport=0
                UDP_dport=0
                UDP_len=0
                UDP_chksum=0





            if j.haslayer(DHCP):
                DHCP_options=str(j[DHCP].options)
                DHCP_options=DHCP_options.replace(",","-")
                if "message" in DHCP_options:
                    x = DHCP_options.find(")")
                    DHCP_options=int(DHCP_options[x-1])

            else:
                DHCP_options=0


            if j.haslayer(BOOTP):
                BOOTP_op=j[BOOTP].op
                BOOTP_htype=j[BOOTP].htype
                BOOTP_hlen=j[BOOTP].hlen
                BOOTP_hops=j[BOOTP].hops
                BOOTP_xid=j[BOOTP].xid
                BOOTP_secs=j[BOOTP].secs
                BOOTP_flags=j[BOOTP].flags
                #if "Flag" in str(BOOTP_flags):BOOTP_flags=str(BOOTP_flags)temp=BOOTP_flags.find("(") BOOTP_flags=int(BOOTP_flags[6:temp-1])
                BOOTP_ciaddr=j[BOOTP].ciaddr
                BOOTP_yiaddr=j[BOOTP].yiaddr
                BOOTP_siaddr=j[BOOTP].siaddr
                BOOTP_giaddr=j[BOOTP].giaddr
                BOOTP_chaddr=j[BOOTP].chaddr
                BOOTP_sname=str(j[BOOTP].sname)
                if BOOTP_sname!="0":
                    BOOTP_sname=1
                else:
                    BOOTP_sname=0
                BOOTP_file=str(j[BOOTP].file)
                if BOOTP_file!="0":
                    BOOTP_file=1
                else:
                    BOOTP_file=0

                BOOTP_options=str(j[BOOTP].options)
                BOOTP_options=BOOTP_options.replace(",","-")
                if BOOTP_options!="0":
                    BOOTP_options=1
                else:
                    BOOTP_options=0
            else:
                BOOTP_op=0
                BOOTP_htype=0
                BOOTP_hlen=0
                BOOTP_hops=0
                BOOTP_xid=0
                BOOTP_secs=0
                BOOTP_flags=0
                BOOTP_ciaddr=0
                BOOTP_yiaddr=0
                BOOTP_siaddr=0
                BOOTP_giaddr=0
                BOOTP_chaddr=0
                BOOTP_sname=0
                BOOTP_file=0
                BOOTP_options=0






            if j.haslayer(DNS):
                DNS_length=j[DNS].length
                DNS_id=j[DNS].id
                DNS_qr=j[DNS].qr
                DNS_opcode=j[DNS].opcode
                DNS_aa=j[DNS].aa
                DNS_tc=j[DNS].tc
                DNS_rd=j[DNS].rd
                DNS_ra=j[DNS].ra
                DNS_z=j[DNS].z
                DNS_ad=j[DNS].ad
                DNS_cd=j[DNS].cd
                DNS_rcode=j[DNS].rcode
                DNS_qdcount=j[DNS].qdcount
                DNS_ancount=j[DNS].ancount
                DNS_nscount=j[DNS].nscount
                DNS_arcount=j[DNS].arcount
                DNS_qd=str(j[DNS].qd).replace(",","-")
                if DNS_qd!="0":
                    DNS_qd=1
                else:
                    DNS_qd=0
                DNS_an=str(j[DNS].an).replace(",","-")
                if DNS_an!="0":
                    DNS_an=1
                else:
                    DNS_an=0
                DNS_ns=str(j[DNS].ns).replace(",","-")
                if DNS_ns!="0":
                    DNS_ns=1
                else:
                    DNS_ns=0
                DNS_ar=str(j[DNS].ar).replace(",","-")
                if DNS_ar!="0":
                    DNS_ar=1
                else:
                    DNS_ar=0
            else:
                DNS_length=0
                DNS_id=0
                DNS_qr=0
                DNS_opcode=0
                DNS_aa=0
                DNS_tc=0
                DNS_rd=0
                DNS_ra=0
                DNS_z=0
                DNS_ad=0
                DNS_cd=0
                DNS_rcode=0
                DNS_qdcount=0
                DNS_ancount=0
                DNS_nscount=0
                DNS_arcount=0
                DNS_qd=0
                DNS_an=0
                DNS_ns=0
                DNS_ar=0





            pdata=[]
            if "TCP" in j:
                pdata = (j[TCP].payload)
            if "Raw" in j:
                pdata = (j[Raw].load)
            elif "UDP" in j:
                pdata = (j[UDP].payload)
            elif "ICMP" in j:
                pdata = (j[ICMP].payload)
            pdata=list(memoryview(bytes(pdata)))

            if pdata!=[]:
                entropy=shannon(pdata)
            else:
                entropy=0
            payload_bytes=len(pdata)

            sport_class=port_class(TCP_sport+UDP_sport)
            dport_class=port_class(TCP_dport+UDP_dport)
            sport23=port_1023(TCP_sport+UDP_sport)
            dport23=port_1023(TCP_dport+UDP_dport)
            sport_bare=TCP_sport+UDP_sport
            dport_bare=TCP_dport+UDP_dport#port_class(TCP_dport+UDP_dport)


            try:Mac=j.src
            except:Mac= j.addr1




            line=[ts,
                  Ether_dst,
            Ether_src,
            IP_src,
            IP_dst,
            pck_size,
            Ether_type,
            LLC_dsap,
            LLC_ssap,
            LLC_ctrl,
            EAPOL_version,
            EAPOL_type,
            EAPOL_len,
            IP_version,
            IP_ihl,
            IP_tos,
            IP_len,
            IP_flags,
            IP_Z,
            IP_MF,
            IP_id,
            IP_chksum,
            IP_DF  ,
            IP_frag,
            IP_ttl,
            IP_proto,
            IP_options,
            ICMP_type,
            ICMP_code,
            ICMP_chksum,
            ICMP_id,
            ICMP_seq,
            ICMP_ts_ori,
            ICMP_ts_rx,
            ICMP_ts_tx,
            ICMP_ptr,
            ICMP_reserved,
            ICMP_length,
            #ICMP_addr_mask,
            ICMP_nexthopmtu,
            ICMP_unused,
            TCP_seq,
            TCP_ack,
            TCP_dataofs,
            TCP_reserved,
            TCP_flags,
            TCP_FIN,
            TCP_SYN,
            TCP_RST,
            TCP_PSH,
            TCP_ACK,
            TCP_URG,
            TCP_ECE,
            TCP_CWR   ,
            TCP_window,
            TCP_chksum,
            TCP_urgptr,
            TCP_options,
            UDP_len,
            UDP_chksum,
            DHCP_options,
            BOOTP_op,
            BOOTP_htype,
            BOOTP_hlen,
            BOOTP_hops,
            BOOTP_xid,
            BOOTP_secs,
            BOOTP_flags,
            BOOTP_sname,
            BOOTP_file,
            BOOTP_options,
            DNS_length,
            DNS_id,
            DNS_qr,
            DNS_opcode,
            DNS_aa,
            DNS_tc,
            DNS_rd,
            DNS_ra,
            DNS_z,
            DNS_ad,
            DNS_cd,
            DNS_rcode,
            DNS_qdcount,
            DNS_ancount,
            DNS_nscount,
            DNS_arcount,
            sport_class,
            dport_class,
            sport23,
            dport23,
            sport_bare,
            dport_bare,
            TCP_sport,
            TCP_dport,
            UDP_sport,
            UDP_dport,
            payload_bytes,
            entropy]

            #print(line)
            line=str(line).replace("[","")
            line=str(line).replace("]","")
            #line=str(line).replace("\',","-")
            line=str(line).replace(", ",",")
            line=str(line).replace("\'","")
            line=str(line).replace("None","0")
            if label: ############################# Label
                Label = df[label_count]
                print(label_count)
            else:
                Label="No_Label"
            ths.write(str(line)+f",{Label}\n")
            label_count+=1
            #kk=line.split(",")
            #print(len(kk))
            #if len(kk)==112:
            #ths.write(line+"\n")

            #else:print(line)
        name=isim.replace("\\","/")
        output=filename.replace("_FE.csv","_WS.csv")

        if " " not in name:
            command=f"tshark -r {name}  -T fields -e _ws.col.Source -e _ws.col.Destination  -e _ws.col.Protocol    -E header=y -E separator=, -E quote=d -E occurrence=f >{output}"
        else:
            command=f"tshark -r \"{name}\" -T fields -e _ws.col.Source -e _ws.col.Destination  -e _ws.col.Protocol    -E header=y -E separator=, -E quote=d -E occurrence=f >\"{output}\""
        os.system(command)
        ths.close()
    if len(file_list)>1:
        merged_csv(i,keyword)

1/40-pcaps/scan-hostport-4-dec.pcap


  3%|▎         | 215/7801 [00:00<00:07, 1051.57it/s]

pcaps/scan-hostport-4-dec.pcap
pcaps/scan-hostport-4-dec.pcap
pcaps/scan-hostport-4-dec.pcap
pcaps/scan-hostport-4-dec.pcap
pcaps/scan-hostport-4-dec.pcap
pcaps/scan-hostport-4-dec.pcap
pcaps/scan-hostport-4-dec.pcap
pcaps/scan-hostport-4-dec.pcap
pcaps/scan-hostport-4-dec.pcap
pcaps/scan-hostport-4-dec.pcap
pcaps/scan-hostport-4-dec.pcap
pcaps/scan-hostport-4-dec.pcap
pcaps/scan-hostport-4-dec.pcap
pcaps/scan-hostport-4-dec.pcap
pcaps/scan-hostport-4-dec.pcap
pcaps/scan-hostport-4-dec.pcap
pcaps/scan-hostport-4-dec.pcap
pcaps/scan-hostport-4-dec.pcap
pcaps/scan-hostport-4-dec.pcap
pcaps/scan-hostport-4-dec.pcap
pcaps/scan-hostport-4-dec.pcap
pcaps/scan-hostport-4-dec.pcap
pcaps/scan-hostport-4-dec.pcap
pcaps/scan-hostport-4-dec.pcap
pcaps/scan-hostport-4-dec.pcap
pcaps/scan-hostport-4-dec.pcap
pcaps/scan-hostport-4-dec.pcap
pcaps/scan-hostport-4-dec.pcap
pcaps/scan-hostport-4-dec.pcap
pcaps/scan-hostport-4-dec.pcap
pcaps/scan-hostport-4-dec.pcap
pcaps/scan-hostport-4-dec.pcap
pcaps/sc

  5%|▌         | 419/7801 [00:00<00:07, 962.03it/s]

pcaps/scan-hostport-4-dec.pcap
pcaps/scan-hostport-4-dec.pcap
pcaps/scan-hostport-4-dec.pcap
pcaps/scan-hostport-4-dec.pcap
pcaps/scan-hostport-4-dec.pcap
pcaps/scan-hostport-4-dec.pcap
pcaps/scan-hostport-4-dec.pcap
pcaps/scan-hostport-4-dec.pcap
pcaps/scan-hostport-4-dec.pcap
pcaps/scan-hostport-4-dec.pcap
pcaps/scan-hostport-4-dec.pcap
pcaps/scan-hostport-4-dec.pcap
pcaps/scan-hostport-4-dec.pcap
pcaps/scan-hostport-4-dec.pcap
pcaps/scan-hostport-4-dec.pcap
pcaps/scan-hostport-4-dec.pcap
pcaps/scan-hostport-4-dec.pcap
pcaps/scan-hostport-4-dec.pcap
pcaps/scan-hostport-4-dec.pcap
pcaps/scan-hostport-4-dec.pcap
pcaps/scan-hostport-4-dec.pcap
pcaps/scan-hostport-4-dec.pcap
pcaps/scan-hostport-4-dec.pcap
pcaps/scan-hostport-4-dec.pcap
pcaps/scan-hostport-4-dec.pcap
pcaps/scan-hostport-4-dec.pcap
pcaps/scan-hostport-4-dec.pcap
pcaps/scan-hostport-4-dec.pcap
pcaps/scan-hostport-4-dec.pcap
pcaps/scan-hostport-4-dec.pcap
pcaps/scan-hostport-4-dec.pcap
pcaps/scan-hostport-4-dec.pcap
pcaps/sc

...

 99%|█████████▉| 72657/73235 [01:35<00:00, 646.94it/s]


pcaps/scan-portos-1-dec.pcap
pcaps/scan-portos-1-dec.pcap
pcaps/scan-portos-1-dec.pcap
pcaps/scan-portos-1-dec.pcap
pcaps/scan-portos-1-dec.pcap
pcaps/scan-portos-1-dec.pcap
pcaps/scan-portos-1-dec.pcap
pcaps/scan-portos-1-dec.pcap
pcaps/scan-portos-1-dec.pcap
pcaps/scan-portos-1-dec.pcap
pcaps/scan-portos-1-dec.pcap
pcaps/scan-portos-1-dec.pcap
pcaps/scan-portos-1-dec.pcap
pcaps/scan-portos-1-dec.pcap
pcaps/scan-portos-1-dec.pcap
pcaps/scan-portos-1-dec.pcap
pcaps/scan-portos-1-dec.pcap
pcaps/scan-portos-1-dec.pcap
pcaps/scan-portos-1-dec.pcap
pcaps/scan-portos-1-dec.pcap
pcaps/scan-portos-1-dec.pcap
pcaps/scan-portos-1-dec.pcap
pcaps/scan-portos-1-dec.pcap
pcaps/scan-portos-1-dec.pcap
pcaps/scan-portos-1-dec.pcap
pcaps/scan-portos-1-dec.pcap
pcaps/scan-portos-1-dec.pcap
pcaps/scan-portos-1-dec.pcap
pcaps/scan-portos-1-dec.pcap
pcaps/scan-portos-1-dec.pcap
pcaps/scan-portos-1-dec.pcap
pcaps/scan-portos-1-dec.pcap
pcaps/scan-portos-1-dec.pcap
pcaps/scan-portos-1-dec.pcap
pcaps/scan-po

 99%|█████████▉| 72760/73235 [01:35<00:00, 762.73it/s]


pcaps/scan-portos-1-dec.pcap
pcaps/scan-portos-1-dec.pcap
pcaps/scan-portos-1-dec.pcap
pcaps/scan-portos-1-dec.pcap
pcaps/scan-portos-1-dec.pcap
pcaps/scan-portos-1-dec.pcap
pcaps/scan-portos-1-dec.pcap
pcaps/scan-portos-1-dec.pcap
pcaps/scan-portos-1-dec.pcap
pcaps/scan-portos-1-dec.pcap
pcaps/scan-portos-1-dec.pcap
pcaps/scan-portos-1-dec.pcap
pcaps/scan-portos-1-dec.pcap
pcaps/scan-portos-1-dec.pcap
pcaps/scan-portos-1-dec.pcap
pcaps/scan-portos-1-dec.pcap
pcaps/scan-portos-1-dec.pcap
pcaps/scan-portos-1-dec.pcap
pcaps/scan-portos-1-dec.pcap
pcaps/scan-portos-1-dec.pcap
pcaps/scan-portos-1-dec.pcap
pcaps/scan-portos-1-dec.pcap
pcaps/scan-portos-1-dec.pcap
pcaps/scan-portos-1-dec.pcap
pcaps/scan-portos-1-dec.pcap
pcaps/scan-portos-1-dec.pcap
pcaps/scan-portos-1-dec.pcap
pcaps/scan-portos-1-dec.pcap
pcaps/scan-portos-1-dec.pcap
pcaps/scan-portos-1-dec.pcap
pcaps/scan-portos-1-dec.pcap
pcaps/scan-portos-1-dec.pcap
pcaps/scan-portos-1-dec.pcap
pcaps/scan-portos-1-dec.pcap
pcaps/scan-po

IndexError: index 72760 is out of bounds for axis 0 with size 72760

# WIRESHARK FEATURES dst,src and Protocol added

In [28]:
name_list=find_the_way('./pcaps','_FE.csv')
for i in tqdm(name_list):
    df=pd.read_csv(i)
    WS_s_d=i.replace("_FE.csv","_WS.csv")
    s_d=pd.read_csv(WS_s_d)
    df["WS_src"]=s_d["_ws.col.Source"]
    df["WS_dst"]=s_d["_ws.col.Destination"]
    df["Protocol"]=s_d["_ws.col.Protocol"]
    y=df["Label"]
    del df["Label"]
    df["Label"]=y
    df.to_csv(i,index=None)
    flag_fixer(i)
    os.remove(WS_s_d)

  0%|          | 0/2 [00:00<?, ?it/s]


FileNotFoundError: [Errno 2] No such file or directory: './pcaps/scan-portos-1-dec_WS.csv'

In [24]:
name_list=find_the_way('./pcaps','_FE.csv')

# NEW FEATURES: dst_IP_diversity, dst_port_diversity, src_IP_diversity IP_add_count

In [25]:
for sayac,i in enumerate (name_list):
    print(f"{sayac+1}/{len(name_list)}-{i}")
    dst_IP_diversity=[]
    dst_port_diversity=[]
    src_IP_diversity=[]
    src_IP_v_dst_IP={}

    IP_add_count=[]
    src_MAC_v_dst_IP={}

    src_IP_v_dst_port={}
    dst_IP_v_src_port={}
    df=pd.read_csv(i)
    count=0
    for j in tqdm(range(len(df))):

        if df['IP_src'][j] in src_IP_v_dst_IP:
            if df['IP_dst'][j] not in src_IP_v_dst_IP[df['IP_src'][j]]:
                src_IP_v_dst_IP[df['IP_src'][j]].append(df['IP_dst'][j])
        else:
            src_IP_v_dst_IP[df['IP_src'][j]]=[]
            src_IP_v_dst_IP[df['IP_src'][j]].append(df['IP_dst'][j])
        dst_IP_diversity.append(len(src_IP_v_dst_IP[df['IP_src'][j]]))

        ###############################################################################

        if df['IP_dst'][j] in dst_IP_v_src_port:
            if df['IP_src'][j] not in dst_IP_v_src_port[df['IP_dst'][j]]:
                dst_IP_v_src_port[df['IP_dst'][j]].append(df['IP_src'][j])
        else:
            dst_IP_v_src_port[df['IP_dst'][j]]=[]
            dst_IP_v_src_port[df['IP_dst'][j]].append(df['IP_src'][j])
        src_IP_diversity.append(len(dst_IP_v_src_port[df['IP_dst'][j]]))

        ###############################################################################
        if df['IP_src'][j] in src_IP_v_dst_port:
            if df['dport_bare'][j] not in src_IP_v_dst_port[df['IP_src'][j]]:
                src_IP_v_dst_port[df['IP_src'][j]].append(df['dport_bare'][j])
        else:
            src_IP_v_dst_port[df['IP_src'][j]]=[]
            src_IP_v_dst_port[df['IP_src'][j]].append(df['dport_bare'][j])
        dst_port_diversity.append(len(src_IP_v_dst_port[df['IP_src'][j]]))
        #################################################################
        if df['Ether_src'][j] in src_MAC_v_dst_IP:
            if df['IP_dst'][j] not in src_MAC_v_dst_IP[df['Ether_src'][j]]:
                src_MAC_v_dst_IP[df['Ether_src'][j]].append(df['IP_dst'][j])
        else:
            src_MAC_v_dst_IP[df['Ether_src'][j]]=[]
            src_MAC_v_dst_IP[df['Ether_src'][j]].append(df['IP_dst'][j])
        IP_add_count.append(len(src_MAC_v_dst_IP[df['Ether_src'][j]]))
        count+=1
        #if count>300000:break
    y=df["Label"]
    del df["Label"]
    df["dst_IP_diversity"]=dst_IP_diversity
    df["dst_port_diversity"]=dst_port_diversity
    df["src_IP_diversity"]=src_IP_diversity
    df["IP_add_count"]=IP_add_count
    df["Label"]=y
    df.to_csv(i,index=None)


1/2-./pcaps/scan-portos-1-dec_FE.csv


100%|██████████| 72743/72743 [00:07<00:00, 9423.84it/s]


2/2-./pcaps/scan-hostport-4-dec_FE.csv


100%|██████████| 7801/7801 [00:00<00:00, 9507.61it/s]


# Sliding Windows Features

In [26]:
outputfolder="FE"
folder(outputfolder)

In [ ]:
files_add=find_the_way('./pcaps','FE.csv')
for i in files_add:
    os.rename(i,i.replace("./pcaps",'./FE'))

In [ ]:
files_add=find_the_way('./FE','FE.csv')
files_add

In [ ]:
def diff(x):
    x = pd.Series(x)
    result=x.diff()
    result=result.fillna(0)
    return result
def mean(x,WS):
    x = pd.Series(x)
    result=x.rolling(WS).mean()
    result=result.fillna(result[WS-1])
    return result
def std(x,WS):
    x = pd.Series(x)
    result=x.rolling(WS).std()
    result=result.fillna(result[WS-1])
    return result

def mean_EW(x):
    x = pd.Series(x)
    result=x.expanding(min_periods=1).mean()
    return result
def std_EW(x):
    x = pd.Series(x)
    result=x.expanding(min_periods=1).std()
    result=result.fillna(0)
    return result
def sum_of_EW(x):
    x = pd.Series(x)
    result=x.expanding(min_periods=1).sum()
    return result

def var(x,WS):
    x = pd.Series(x)
    result=x.rolling(WS).var()
    result=result.fillna(result[WS-1])
    return result
def flagsum(x):
    total_f=0
    result=[]
    for i in x:
        if i==1:
            total_f+=1
        result.append(total_f)
    return result
def portsum(x):
    total_ports=[]
    result=[]
    for i in x:
        if i not in total_ports:
            total_ports.append(i)
        result.append(len(total_ports))
    return result

In [ ]:
n=['ts',"WS_src","WS_dst",
 'pck_size',
 #'Ether_dst',
 #'Ether_src','IP_src',
 #'IP_dst',
   'UDP_sport',
 'UDP_dport','TCP_sport',
 'TCP_dport',
#'IP_src',
 #'IP_dst',
'TCP_FIN',
 'TCP_SYN',
 'TCP_RST',
 'TCP_PSH',
 'TCP_ACK',
 'TCP_URG',
 'TCP_ECE',
 'TCP_CWR',
 'TCP_window',"payload_bytes","entropy","Label"]

In [ ]:
for j in files_add:
    df=pd.read_csv(j)
    break

In [ ]:
outputfolder="SW"
folder(outputfolder)

for windows_size in [2,6,9]:
    for j in files_add:
        print(j[5:])
        df=pd.read_csv(j)#,usecols=n)

        df["sport"]=df["UDP_sport"]+df["TCP_sport"]
        df["dport"]=df["UDP_dport"]+df["TCP_dport"]

        del df["UDP_sport"]
        del df["TCP_sport"]
        del df["UDP_dport"]
        del df["TCP_dport"]

        df.to_csv("temp.csv")


        df=pd.read_csv("temp.csv")
        #df["ID"] = df["Ether_src"] +"@" +df["IP_src"] +"====>"+df["Ether_dst"]+ "@"+ df["IP_dst"]
        df["ID"] = df["WS_src"]+"=>"+df["WS_dst"]
        #del df["Ether_src"]
        #del df["IP_src"]
        #del df["Ether_dst"]
        #del df["IP_dst"]
        del df["WS_src"]
        del df["WS_dst"]

        cols=[ 'TCP_FIN',
     'TCP_SYN',
     'TCP_RST',
     'TCP_PSH',
     'TCP_ACK',
     'TCP_URG',
     'TCP_ECE',
     'TCP_CWR', 'Label', 'ID', 'pck_size_diff', 'pck_size_mean',
               'pck_size_std',  'ts_diff', 'ts_mean', 'ts_std',
                'TCP_window_diff', 'TCP_window_mean', 'TCP_window_std',"payload_bytes_diff","payload_bytes_mean","payload_bytes_std",
                'entropy_diff',
         'entropy_mean',
         'entropy_std',
'pck_size_mean_EW','pck_size_std_EW',
'ts_mean_EW', 'ts_std_EW',
'TCP_window_mean_EW','TCP_window_std_EW',
'payload_bytes_mean_EW','payload_bytes_std_EW',
'entropy_mean_EW','entropy_std_EW',
'ts_sum_of_EW', 'pck_size_sum_of_EW',
'payload_bytes_sum_of_EW','entropy_sum_of_EW','TCP_window_sum_of_EW',"Label"]
        df[df.columns[-1]]=df[df.columns[-1]].astype(str)
        IDS=sorted(list(df[df.columns[-1]].unique()))
        func_sw=[mean,std] #functions
        func_ew=[diff,mean_EW,std_EW,sum_of_EW] #functions
        func_name_sw=["mean","std"]
        func_name_ew=["diff","mean_WE","std_WE","sum_of_EW"]

        f_list=["pck_size","ts",'TCP_window',"payload_bytes","entropy"]#, "dport","sport"     'TCP_SYN', 'TCP_ACK'    'TCP_FIN', 'TCP_SYN', 'TCP_RST', 'TCP_PSH', 'TCP_ACK', 'TCP_URG', 'TCP_ECE', 'TCP_CWR',
        fark=[#'pck_size_diff',
         'pck_size_mean',
         'pck_size_std',
         #'ts_diff',
         'ts_mean',
         'ts_std',
         #'TCP_window_diff',
         'TCP_window_mean',
         'TCP_window_std',#"payload_bytes_diff",
          "payload_bytes_mean","payload_bytes_std",
         #'entropy_diff',
         'entropy_mean',
         'entropy_std']
        WS=windows_size
        flag=1
        for i in tqdm(IDS):
            smaller = df[df["ID"]==i]
            smaller=smaller.reset_index()
            del smaller["index"]
            # smaller["ts"]=np.array(diff(smaller["ts"].values)) older way
            #print( smaller["ts"])
            smaller["ts"]=smaller["ts"].values-smaller["ts"].values[0]
            #print( smaller["ts"])
            for ii in f_list:
                for jjj,iii in enumerate (func_ew):
                    name=str(ii)+"_"+str(func_name_ew[jjj])
                    smaller[name]=iii(smaller[ii])
            if len(smaller)>=WS:
                for ii in f_list:
                    for jjj,iii in enumerate (func_sw):
                        name=str(ii)+"_"+str(func_name_sw[jjj])
                        smaller[name]=iii(smaller[ii],WS)
            else:
                for ii in fark:
                        smaller[ii]=0
            smaller["dport_sum"]=portsum(smaller["dport"].values)
            smaller["sport_sum"] =portsum(smaller["sport"].values)
            smaller['TCP_FIN_sum'  ]=flagsum (smaller[ 'TCP_FIN'  ].values)
            smaller['TCP_SYN_sum'  ]=flagsum (smaller['TCP_SYN'  ].values)
            smaller['TCP_RST_sum'  ]=flagsum (smaller['TCP_RST'  ].values)
            smaller['TCP_PSH_sum'  ]=flagsum (smaller['TCP_PSH'  ].values)
            smaller['TCP_ACK_sum'  ]=flagsum (smaller['TCP_ACK'  ].values)
            smaller['TCP_URG_sum'  ]=flagsum (smaller['TCP_URG'  ].values)
            smaller['TCP_ECE_sum'  ]=flagsum (smaller['TCP_ECE'  ].values)
            smaller['TCP_CWR_sum'  ]=flagsum (smaller['TCP_CWR'  ].values)
            #del smaller["dport"]
            #del smaller["sport"]
            #del smaller["ID"]


            smaller["TCP_FIN_ratio"]=smaller["TCP_FIN"]/(smaller["TCP_FIN_sum"]+10e-20)
            smaller["TCP_SYN_ratio"]=smaller["TCP_SYN"]/(smaller["TCP_SYN_sum"]+10e-20)
            smaller["TCP_RST_ratio"]=smaller["TCP_RST"]/(smaller["TCP_RST_sum"]+10e-20)
            smaller["TCP_PSH_ratio"]=smaller["TCP_PSH"]/(smaller["TCP_PSH_sum"]+10e-20)
            smaller["TCP_ACK_ratio"]=smaller["TCP_ACK"]/(smaller["TCP_ACK_sum"]+10e-20)
            smaller["TCP_URG_ratio"]=smaller["TCP_URG"]/(smaller["TCP_URG_sum"]+10e-20)
            smaller["TCP_ECE_ratio"]=smaller["TCP_ECE"]/(smaller["TCP_ECE_sum"]+10e-20)
            smaller["TCP_CWR_ratio"]=smaller["TCP_CWR"]/(smaller["TCP_CWR_sum"]+10e-20)
            col_list= ["TCP_FIN_sum","TCP_SYN_sum","TCP_RST_sum","TCP_PSH_sum","TCP_ACK_sum","TCP_URG_sum","TCP_ECE_sum","TCP_CWR_sum"]
            smaller['sum'] = smaller[col_list].sum(axis=1)
            smaller["TCP_FIN_SR"]=smaller["TCP_FIN"]/(smaller["sum"]+10e-20)
            smaller["TCP_SYN_SR"]=smaller["TCP_SYN"]/(smaller["sum"]+10e-20)
            smaller["TCP_RST_SR"]=smaller["TCP_RST"]/(smaller["sum"]+10e-20)
            smaller["TCP_PSH_SR"]=smaller["TCP_PSH"]/(smaller["sum"]+10e-20)
            smaller["TCP_ACK_SR"]=smaller["TCP_ACK"]/(smaller["sum"]+10e-20)
            smaller["TCP_URG_SR"]=smaller["TCP_URG"]/(smaller["sum"]+10e-20)
            smaller["TCP_ECE_SR"]=smaller["TCP_ECE"]/(smaller["sum"]+10e-20)
            smaller["TCP_CWR_SR"]=smaller["TCP_CWR"]/(smaller["sum"]+10e-20)

            smaller["TCP_FIN_R"]=smaller["TCP_FIN_sum"]/(smaller["sum"]+10e-20)
            smaller["TCP_SYN_R"]=smaller["TCP_SYN_sum"]/(smaller["sum"]+10e-20)
            smaller["TCP_RST_R"]=smaller["TCP_RST_sum"]/(smaller["sum"]+10e-20)
            smaller["TCP_PSH_R"]=smaller["TCP_PSH_sum"]/(smaller["sum"]+10e-20)
            smaller["TCP_ACK_R"]=smaller["TCP_ACK_sum"]/(smaller["sum"]+10e-20)
            smaller["TCP_URG_R"]=smaller["TCP_URG_sum"]/(smaller["sum"]+10e-20)
            smaller["TCP_ECE_R"]=smaller["TCP_ECE_sum"]/(smaller["sum"]+10e-20)
            smaller["TCP_CWR_R"]=smaller["TCP_CWR_sum"]/(smaller["sum"]+10e-20)



            if flag:
                smaller.to_csv(j[5:],header=True,index=False)
                flag=0
            else:
                smaller.to_csv(j[5:],header=False,index=False,mode="a")
        df=pd.read_csv(j[5:])
        features=df.columns
        count=[]
        bos=[]

        #for ii,i in enumerate (features):
            #a=pd.DataFrame(df.groupby(i).size())
            #if a.shape[0]>1:
                #print(a)
               # count.append(i)
                #print("________________________________________________________________\n\n")
            #else:
                #bos.append(i)
        df=df.sort_values(by='Unnamed: 0')

        y=df["Label"]
        #for i in bos:
            #print(i)
            #del df[i]
        del df["Label"]
        del df['Unnamed: 0']
        df["Label"]=y
        last_name=f'./{outputfolder}/last_{str(WS)}_{j[5:]}'
        df.to_csv(last_name,index=None)
        print(WS,"-------------------------------------------------------------------------------")


# MERGE SW FILES

In [ ]:
files_add=find_the_way('./SW','FE.csv')
unique_file_names=[]
for i in files_add:
    bas=i.replace('_', '@', 1).find('_')+1
    if i[bas:] not in unique_file_names:
        unique_file_names.append(i[bas:])
for i in tqdm(unique_file_names):
    files_add=find_the_way('./SW',i)
    flag=1
    for ii in files_add:
        WS=ii.split("_")[1]
        #print(WS)
        new_cols={'pck_size_mean':f'pck_size_mean_{WS}',
        'pck_size_std':f'pck_size_std_{WS}',
        'ts_mean':f'ts_mean_{WS}',
        'ts_std':f'ts_std_{WS}',
        'TCP_window_mean':f'TCP_window_mean_{WS}',
        'TCP_window_std':f'TCP_window_std_{WS}',
        'payload_bytes_mean':f'payload_bytes_mean_{WS}',
        'payload_bytes_std':f'payload_bytes_std_{WS}',
        'entropy_mean':f'entropy_mean_{WS}',
        'entropy_std':f'entropy_std_{WS}'}

        #print(new_cols)
        if flag:
             df=pd.read_csv(ii)
             df.rename(columns = new_cols, inplace = True)
             flag=0
        else:
            adding=pd.read_csv(ii,usecols=list(new_cols.keys()))
            adding.rename(columns = new_cols, inplace = True)
            df = pd.concat([df, adding], axis=1)
        os.remove(ii)
    temp=i.replace("_FE.","_SW.")
    temp=f"./SW/{temp}"
    y=df["Label"]
    del df["Label"]
    df["Label"]=y
    df.to_csv(temp,index=False)
    #print(list(df.columns))
    #print(f"{WS}\n\n\n\n")

